<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 데이터 로드 & 상위 데이터 추출

- 각 배색에 대한 조화도 평점을 기준으로, 내림차순 정렬하여 상위 3000개를 데이터 프레임으로 추출 완료함
    - 상위 3000개 데이터의 조화도 평균 평점 : 4.35, 표준편차 : 0.26

In [ ]:
!git clone https://github.com/YNNJN/colorPool_Data.git

Cloning into 'colorPool_Data'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 52 (delta 26), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [ ]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Sep 16 16:29 sample_data
drwxr-xr-x 4 root root 4096 Sep 22 09:49 colorPool_Data


In [ ]:
# connect to google dirve

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

rgbData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_data.csv', names=['R1', 'R2', 'R3', 'R4', 'R5', 'G1', 'G2', 'G3', 'G4', 'G5', 'B1', 'B2', 'B3', 'B4', 'B5' ])
ratingData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_targets.csv', names=['rating'])
idData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_ids.csv', names=['id'])
nameData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/kulerData_name.csv', names=['name'])

rgbRatingData = pd.concat([rgbData, ratingData, idData, nameData], axis=1)

In [ ]:
df = pd.DataFrame(rgbRatingData)
df_sorted = df.sort_values(by=['rating'], ascending=[False])
df_extracted = df_sorted[0:3000]
df_rgbExtracted = df_extracted.drop(df_extracted.columns[-1], axis=1)
df_rgbExtracted = df_rgbExtracted.drop(df_extracted.columns[-2], axis=1)
df_rgbExtracted = df_rgbExtracted.drop(df_extracted.columns[-3], axis=1)

df_rgbExtracted

# df_extracted.to_csv('gdrive/My Drive/colorPOOL/kulerData/matrixData.csv')

,R1,R2,R3,R4,R5,G1,G2,G3,G4,G5,B1,B2,B3,B4,B5
44985,0.43922,0.619610,0.45490,0.27843,0.388240,0.286270,0.52157,0.47843,0.27451,0.30196,0.172550,0.270590,0.341180,0.196080,0.16471
37404,0.05098,0.949020,0.74902,0.68235,0.643140,0.050980,0.94902,0.85098,0.85098,0.85098,0.050980,0.949020,0.721570,0.639220,0.58824
36535,0.93333,0.850980,0.67059,0.42745,0.317650,0.568630,0.33333,0.74902,0.65098,0.55686,0.058824,0.145100,0.082353,0.090196,0.65098
4099,0.76863,0.039216,0.80392,0.40000,0.000000,0.000000,0.50980,0.00000,0.25490,0.00000,0.000000,0.756860,0.509800,0.125490,0.00000
4091,1.00000,0.823530,0.88627,0.67059,0.403920,0.984310,0.76863,0.44314,0.32157,0.21569,0.839220,0.725490,0.215690,0.196080,0.13725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31263,0.80000,0.980390,0.67451,0.34510,0.164710,0.325490,1.00000,1.00000,0.80000,0.52549,0.513730,0.580390,0.874510,0.623530,0.60000
31270,0.65098,0.949020,0.94902,0.15294,0.372550,0.545100,0.94902,0.91373,0.17255,0.45098,0.011765,0.949020,0.427450,0.188240,0.40000
31278,0.85098,0.850980,0.94902,0.47451,0.011765,0.015686,0.40392,0.62353,0.65098,0.58824,0.015686,0.015686,0.019608,0.090196,0.65098
31279,0.14902,0.549020,0.85098,0.94902,0.949020,0.003922,0.15294,0.41569,0.56863,0.88235,0.003922,0.011765,0.160780,0.239220,0.55294


# 1. 표본 색채 설정 & 색차 허용치 보정
- 표본 색채 설정 이유
    - Kuler에서 활용되는 색채들 중 서로 비슷한 색들을 근사하여 분석하기 위해, 인지적으로 균등한 표본 색채를 설정함
- 색차 허용치 보정 이유
    - 색차 표본간의 거리를 사람이 인식하는 색차와 균등하게 보정하기 위함

#### McDonald의 색차 계산식 이용
- 이유 : CIE L*a*b 색공간 내에서의 기하학적 거리 ≠ 사람의 인지적 색차
    - 방법 : 색차허용치 = 색공간 내 기하학적 거리/(1+0.022(채도값)) = 6.35
    - 의의 : 색채 표본간의 거리를 사람이 인식하는 색차와 균등하게 보정 완료
- 진행과정 :

## 1) 데이터 변환
- RGB → XYZ → CIE L*a*b

In [ ]:
# 데이터프레임 구조 변환 3000*15 => 15000*3

df1=df_rgbExtracted.iloc[:,[0,5,10]].reset_index(drop=True).rename(columns={"R1":'R',"G1":'G',"B1":'B'})
df2=df_rgbExtracted.iloc[:,[1,6,11]].reset_index(drop=True).rename(columns={"R2":'R',"G2":'G',"B2":'B'})
df3=df_rgbExtracted.iloc[:,[2,7,12]].reset_index(drop=True).rename(columns={"R3":'R',"G3":'G',"B3":'B'})
df4=df_rgbExtracted.iloc[:,[3,8,13]].reset_index(drop=True).rename(columns={"R4":'R',"G4":'G',"B4":'B'})
df5=df_rgbExtracted.iloc[:,[4,9,14]].reset_index(drop=True).rename(columns={"R5":'R',"G5":'G',"B5":'B'})

df_all=pd.concat([df1,df2,df3,df4,df5], axis=0).reset_index(drop=True)
df_all

,R,G,B
0,0.439220,0.28627,0.172550
1,0.050980,0.05098,0.050980
2,0.933330,0.56863,0.058824
3,0.768630,0.00000,0.000000
4,1.000000,0.98431,0.839220
...,...,...,...
14995,0.164710,0.52549,0.600000
14996,0.372550,0.45098,0.400000
14997,0.011765,0.58824,0.650980
14998,0.949020,0.88235,0.552940


In [ ]:
!pip install colormath

  Created wheel for colormath: filename=colormath-3.0.0-cp36-none-any.whl size=39392 sha256=365b6fec38fbc6f04a081a5685d8daed329101f99063a79e4e4c586170b6e80b
  Stored in directory: /root/.cache/pip/wheels/20/49/7b/9635bccd6136cc713f623caa85ced89c0c63878f8b2a58fe99
Successfully built colormath


In [ ]:
# RGB → XYZ

from colormath.color_objects import XYZColor, sRGBColor
from colormath.color_conversions import convert_color

x=[]
y=[]
z=[]

for i in range(len(df_all)):
    rgb = sRGBColor(df_all['R'][i], df_all['G'][i], df_all['B'][i])
    xyz = convert_color(rgb, XYZColor, target_illuminant='d50') 
    x.append(xyz.xyz_x)
    y.append(xyz.xyz_y)
    z.append(xyz.xyz_z)

In [ ]:
xyz=pd.DataFrame({"X":x,"Y":y,"Z":z})
xyz.head()

,X,Y,Z
0,0.099912,0.085339,0.026717
1,0.003881,0.004025,0.003322
2,0.482525,0.393500,0.042819
3,0.240701,0.122816,0.007690
4,0.903736,0.954817,0.587924


In [ ]:
!pip install -q colour-science

     |████████████████████████████████| 1.6MB 6.7MB/s 


In [ ]:
# XYZ → CIE Lab

import colour

l=[]
a=[]
b=[]

for i in range(len(xyz)):
    lab = colour.XYZ_to_Lab([xyz['X'][i], xyz['Y'][i], xyz['Z'][i]])

    l.append(lab[0])
    a.append(lab[1])
    b.append(lab[2])

In [ ]:
lab = pd.DataFrame({"L":l,"a":a,"b":b})
lab

,L,a,b
0,35.070886,15.841340,29.939625
1,3.635469,0.226806,1.517953
2,69.004011,32.474404,78.551124
3,41.660251,67.802908,60.830670
4,98.225945,-0.683832,34.091692
...,...,...,...
14995,51.309308,-21.445291,-6.572725
14996,46.501122,-8.569418,13.739010
14997,56.284027,-28.530707,-5.793954
14998,89.555184,-0.194870,55.466495


## 2) 데이터 양자화


In [ ]:
# quantize

def quant(x):
    if x % 5 == 0:
        return int(x)
    else:
        return int(int((x + 5) / 5) * 5)

quant_l = []
quant_a = []
quant_b = []

for i in range(len(lab)):
    quant_l.append(quant(lab.L[i]))
    quant_a.append(quant(lab.a[i]))
    quant_b.append(quant(lab.b[i]))

In [ ]:
quant_lab = pd.DataFrame({"L" : quant_l, "a" : quant_a, "b": quant_b})
quant_lab

,L,a,b
0,40,20,30
1,5,5,5
2,70,35,80
3,45,70,65
4,100,0,35
...,...,...,...
14995,55,-15,0
14996,50,0,15
14997,60,-20,0
14998,90,0,60


In [ ]:
# 적정 dEh 값과 비교하기 위한 테스트 진행
# Cab = (a ** 2 + b ** 2) ** 1/2
# dEh = dE / (1 + 0.022 * Cab)

dE = []

for i in range(len(lab)):
    previous = [lab.L[i], lab.a[i], lab.b[i]]
    next = [quant_lab.L[i], quant_lab.a[i], quant_lab.b[i],]

    de = colour.delta_E(previous, next, method='CIE 2000')
    dE.append(de)

In [ ]:
cab = []

for i in range(len(lab)):
    cal = (lab.a[i] ** 2 + lab.b[i] ** 2) ** 1/2
    cab.append(cal)

In [ ]:
tmp = pd.DataFrame({"dE": dE, "Cab" : cab})
tmp

,dE,Cab
0,5.069017,573.664600
1,6.630312,1.177812
2,1.347346,3612.433003
3,3.300368,4148.802352
4,1.208111,581.355545
...,...,...
14995,6.992094,251.550606
14996,10.306413,131.097656
14997,6.630125,423.785562
14998,1.297228,1538.284996


In [ ]:
deh = []
for i in range(len(tmp)):
    cal = tmp.dE[i] / (1 + 0.022 * tmp.Cab[i])
    deh.append(cal)

In [ ]:
deh = pd.DataFrame(data=deh, columns=['dEh'])
deh

,dEh
0,0.372158
1,6.462847
2,0.016743
3,0.035767
4,0.087609
...,...
14995,1.070091
14996,2.653455
14997,0.642250
14998,0.037231


In [ ]:
# 평균
deh.mean() # 0.914265

dEh    0.955119
dtype: float64

In [ ]:
# 분산
deh.var() # 1.934336

dEh    2.040061
dtype: float64

## 3) 데이터 양자화 2
- 양자화 간격을 3으로 조정함
    - dE의 값을 5 이하로 하는 dL, da, db 의 값을 추산한 결과로 간격인 3을 결정함
- 인지적으로 균등한 색채 이론 참고
    - [Perceptual Color Space](http://cs.haifa.ac.il/hagit/courses/ist/Lectures/IST05_ColorLAB.pdf)
- 양자화 후 dEh 값을 역으로 계산하여 양자화 신뢰도 검증함
    - dEh 값은 평균 0.591941, 분산은 0.910121
    - 데이터의 신뢰도 검증 완료
- 양자화 한 색채로 테마를 구성했을 때의 시각적 유사도 또한 육안으로 검증하는 과정을 거쳤음

In [ ]:
# quantize

def quant2(x):
    if x % 3 == 0:
        return int(x)
    else:
        return int(int((x + 3) / 3) * 3)

quant2_l = []
quant2_a = []
quant2_b = []

for i in range(len(lab)):
    quant2_l.append(quant2(lab.L[i]))
    quant2_a.append(quant2(lab.a[i]))
    quant2_b.append(quant2(lab.b[i]))

In [ ]:
quant2_lab = pd.DataFrame({"L" : quant2_l, "a" : quant2_a, "b": quant2_b})
quant2_lab
quant2_lab.to_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')

In [ ]:
dE = []

for i in range(len(lab)):
    previous = [lab.L[i], lab.a[i], lab.b[i]]
    next = [quant2_lab.L[i], quant2_lab.a[i], quant2_lab.b[i],]

    de = colour.delta_E(previous, next, method='CIE 2000')
    dE.append(de)

In [ ]:
cab = []

for i in range(len(lab)):
    cal = (lab.a[i] ** 2 + lab.b[i] ** 2) ** 1/2
    cab.append(cal)

In [ ]:
tmp = pd.DataFrame({"dE": dE, "Cab" : cab})
tmp

,dE,Cab
0,1.684818,573.664600
1,4.177201,1.177812
2,2.371840,3612.433003
3,0.685026,4148.802352
4,1.018892,581.355545
...,...,...
14995,4.028362,251.550606
14996,6.240241,131.097656
14997,4.482813,423.785562
14998,0.532396,1538.284996


In [ ]:
deh = []
for i in range(len(tmp)):
    cal = tmp.dE[i] / (1 + 0.022 * tmp.Cab[i])
    deh.append(cal)

In [ ]:
deh2 = pd.DataFrame(data=deh, columns=['dEh'])
deh2

,dEh
0,0.123696
1,4.071696
2,0.029474
3,0.007424
4,0.073887
...,...
14995,0.616512
14996,1.606592
14997,0.434243
14998,0.015280


In [ ]:
# 평균
deh2.mean() # 0.591941

dEh    0.618456
dtype: float64

In [ ]:
# 분산
deh2.var() # 0.910121

dEh    0.962554
dtype: float64